Quantum State Preparation - QSP

In [87]:
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit.quantum_info import Statevector
from math import sqrt
from numpy.linalg import svd

In [118]:
def preparacao_de_estados(n):
    
    if n % 2 != 0:
        raise ValueError("O número de qubits deve ser par.")
    
    qc = QuantumCircuit(n)
    k = n//2

    #-------------------#
    # Fase 1
          
    # Dividir o circuito em 2 grupos de qubits, cada um com k qubits
    # a primeira metade com amplitude uniforme e a segunda metade com amplitude 0
    alpha = [1/sqrt((2**n)/2)] * (2**(n-1))
    statevector = alpha + ([0] * (2**(n-1)))
    
    # Inicializar o circuito com o estado desejado
    qc.initialize(statevector, qc.qubits)

    #-------------------#
    # Fase 2

    # Aplicando cnots para criar um emaranhamento entre os qubits
    for i in range(k):
        qc.cx(i, i+k)
    
    #-------------------#
    # Fase 3 & 4
    # Realizar a decomposição em valores singulares (SVD)

    # Coletar a matriz unitaria do circuito quantico
    unitary = Statevector.from_instruction(qc).data.reshape(2**k, 2**k)
    print(unitary)

    # Realizar a SVD
    U, S, Vh = svd(unitary)

    # Aplicar os componentes do SVD ao circuito quantico
    for i in range(k):
        qc.ry(2 * S[i], i)
        qc.unitary(U, list(range(k)), label='U')
        qc.unitary(Vh, list(range(k, n)), label='Vh')

    #-------------------#

    return qc

# Exemplo de uso
n = 4
qc = preparacao_de_estados(n)
qc.draw()

[[0.35355339+0.j 0.35355339+0.j 0.        +0.j 0.        +0.j]
 [0.35355339+0.j 0.35355339+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.35355339+0.j 0.35355339+0.j]
 [0.        +0.j 0.        +0.j 0.35355339+0.j 0.35355339+0.j]]


»
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                             ├»
«     │                                                                                              │»
«q_1: ┤1                                                                                             ├»
«     │  Initialize(0.35355,0.35355,0.35355,0.35355,0.35355,0.35355,0.35355,0.35355,0,0,0,0,0,0,0,0) │»
«q_2: ┤2                                                                                             ├»
«     │                                                                                              │»
«q_3: ┤3                                                                                             ├»
«     └──────────────────────────────────────────────────────────────────────────────────────────────┘»
«          ┌────────────┐ ┌────┐              ┌────┐
«q_0: ──■──┤ Ry(1.4142) ├─┤0   ├──────────────┤0   ├
«       │  └────────────┘ │  U │┌────────────┐│  U │
«q_1: ──┼────────■────────┤1   ├┤ Ry(1.4142) ├┤1   ├
«     ┌─┴─┐      │       ┌┴────┤└──┬─────┬───┘└────┘
«q_2: ┤ X ├──────┼───────┤0    ├───┤0    ├──────────
«     └───┘    ┌─┴─┐     │  Vh │   │  Vh │          
«q_3: ─────────┤ X ├─────┤1    ├───┤1    ├──────────
«              └───┘     └─────┘   └─────┘

In [120]:
qc.measure_all()

# initialization of the sampler
sampler = Sampler()

job = sampler.run([qc])
result = job.result()
print(f" > Counts: {result[0].data.meas.get_counts()}")

 > Counts: {'1011': 477, '0001': 529, '0000': 9, '1010': 9}


Quantum Error Correction - QEC